In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
res=requests.get("https://www.gvm.com.tw/category/money?page=")

soup=BeautifulSoup(res.text, "html.parser")

In [3]:
res

<Response [200]>

In [4]:
soup

<!DOCTYPE html>
 <html> <head> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <title>金融理財 | 遠見雜誌 - 前進的動力</title> <meta charset="utf-8"/> <meta content="width=device-width, initial-scale=1.0, maximum-scale=1, minimum-scale=1, initial-scale=1" name="viewport"/> <meta content="金融理財 | 遠見雜誌 - 前進的動力" name="title"/> <meta content="投資理財、保險規劃與退休理財等，理財面相議題。" name="description"/> <meta content="zh_TW" property="og:locale"/> <meta content="遠見雜誌 - 前進的動力" property="og:site_name"/> <meta content="website" property="og:type"/> <meta content="金融理財 | 遠見雜誌 - 前進的動力" property="og:title"/> <meta content="https://www.gvm.com.tw/category/money" property="og:url"/> <meta content="https://imgs.gvm.com.tw/images/blog/0_20150720001_1.jpg" property="og:image"/> <meta content="620" property="og:image:width"/> <meta content="434" property="og:image:height"/> <meta content="投資理財、保險規劃與退休理財等，理財面相議題。" property="og:description"/> <meta content="https://www.facebook.com/gv.monthly/" property="article

In [111]:
b=soup.select(".article-list__pagination a")
b

[<a href="https://www.gvm.com.tw/category/money?page=2"><span aria-hidden="true"><i aria-hidden="true" class="fa fa-chevron-left"></i></span></a>,
 <a href="https://www.gvm.com.tw/category/money?page=">1</a>,
 <a href="https://www.gvm.com.tw/category/money?page=2">2</a>,
 <a>3</a>,
 <a href="https://www.gvm.com.tw/category/money?page=4">4</a>,
 <a href="https://www.gvm.com.tw/category/money?page=5">5</a>,
 <a href="https://www.gvm.com.tw/category/money?page=4"><span aria-hidden="true"><i aria-hidden="true" class="fa fa-chevron-right"></i></span></a>]

In [32]:
url="https://www.gvm.com.tw/category/money?page="
dates=[]
titles=[]
links=[]
i = 2

for times in range(20):
    res = requests.get(url)
    soup=BeautifulSoup(res.text, "html.parser")
    articles=soup.select(".article-list-item__intro h3")
    link=soup.select(".article-list-item__img a")
    time = soup.select(".time")
    next_url = "https://www.gvm.com.tw/category/money?page=" + str(times + i)
    
    url=next_url
    for each_date in time:
        dates.append(each_date.text)
    for each_title in articles:
        titles.append(each_title.text)
    for each_link in link:
        links.append(each_link["href"])

df = pd.DataFrame(
{
    '日期':dates,
    '標題':titles,
    '內容連結':links
})

#df.to_csv('遠見爬蟲.csv',encoding='utf_8')

df

,日期,標題,內容連結
0,2021-06-02,吳東亮終於可分手彰銀！「台新金」將以三招擺脫16年併購包袱,https://www.gvm.com.tw/article/79966
1,2021-06-02,本土疫情爆發下，首當其衝的餐飲、百貨股能翻身嗎？,https://www.gvm.com.tw/article/79950
2,2021-06-01,ESG意識抬頭，近7成台灣女性想用投資讓世界變美好,https://www.gvm.com.tw/article/79815
3,2021-06-01,金管會點頭！台新金併保德信核准，承諾六年內處分彰銀持股,https://www.gvm.com.tw/article/79945
4,2021-06-01,全球經濟邁向多速復甦 價值回歸投資成王道,https://www.gvm.com.tw/article/79636
...,...,...,...
595,2019-11-15,這張保單比儲蓄險還紅？保費暴增16.55倍，該跟風買嗎？,https://www.gvm.com.tw/article/69407
596,2019-11-14,提升市民休憩品質 邁向國際級運動城市,https://www.gvm.com.tw/article/69410
597,2019-11-14,抓到了！兩大隱形殺手讓你的錢愈變愈薄,https://www.gvm.com.tw/article/69367
598,2019-11-13,「地表最強分析師」日常生活曝光：這就是我們與有錢人的距離,https://www.gvm.com.tw/article/69397


In [46]:
df1=df.drop(df.tail().index)
df1

,日期,標題,內容連結


In [39]:
cleaned = []

s_list=[]

for article_url in df1["內容連結"]:
    s_list.clear()
    res2 = requests.get(article_url)
    soup2 = BeautifulSoup(res2.text, "html.parser")
    temp_content = soup2.select(".article-content p")
    for data in temp_content:
        if data.text.__contains__("圖／") or data.text.__contains__("不代表本社立場") or data.text.__contains__("會員") or data.text.__contains__("立即閱讀"):
            continue
        s_list.append(data.text.strip())
    list_copy = list(s_list)  #把list另存新檔
    cleaned.append("".join(list_copy)) 

cleaned

['多年來，台新金控董事長吳東亮曾設法啟動併購、更名，甚至不惜全盤放棄彰銀，卻始終無法如願，無論是財政部、台新金、彰銀甚至是員工，都沒有人是贏家，甚至，影響外資投資台灣的意願。等了將近16年，每每提起彰銀二字，就如同魚刺哽在喉嚨、有苦說不出的吳東亮，終於有望擺脫包袱，納進壽險作為第三引擎，全力奮追國泰、富邦、中信、開發等幾大民營金控了！故事的開頭，在2005年的7月22日下午兩點鐘，為了促進官股民營化，改善沉重的財務沉痾，彰化銀行特別股標售案開標，而台新金正是出價最高者，當天台新金控董事長吳東亮一得知喜訊，馬上興奮地打電話向太座彭雪芬報喜，因為這起標售案附有經營權的移轉，這意味著台新金將一舉壯大版圖規模，甚至打開國際市場。豈料，這個在二次金改政策目標下，由財政部主導整頓彰銀財務的標售案，卻在接下來的年頭裡，由於政黨輪替及雙方對當年標售條件的解讀有歧異，台新金未能如願併購彰銀，更因這筆鉅額投資，而導致雙重槓桿比率過高，限制到台新金自身的資本規劃與長期發展，更曾因此不符合參股中國金融機構的資格。台新金一度想合併彰銀，甚至連名字都可以放棄。但近年來每三年展開的董監改選大戰，總是備受外界矚目，多位產官學者都曾出示意見；而現在，台新金和財政部的官司未了，問題就卡在，官民各有立場，不能輕易讓步。然而，在特許行業的金融業裡，官民爭訟，可謂是破天荒史上頭一遭。然而，這一切，都在今年6月1日，宣布可望解套了。去年8月11日，台新金宣布標得保德信人壽，預計以55到85億元台幣收購。然而，台新金是彰銀最大單一股東，第二大股東財政部持股則有12.19%，這起併購案和彰銀環環相扣。台新金有意出售彰銀股權以籌措資金，強化體質，但這筆龐大持股，由誰承接，又該怎麼接，台新金、財政部和金管會頻繁地進行討論。6月1日下午4點，金管會在例行記者會上，宣布有條件地放行台新金併購保德信人壽，台新金將以出售彰化銀行股票方式，籌措買保德信人壽的一半交割股款，銀行局主秘童政彰表示，依據《金控法》規定，台新金一旦開始出售彰銀股票，即不再視彰銀為子公司，手中持有的23.4億股彰銀要全數交由銀行保管，未來全數出脫。當天5點半，台新金總經理林維俊先在證交所發布重訊記者會，6點15分緊接著召開金控線上記者會，他語氣和緩地宣布，將以六年時間全數出清彰銀股權，強調「一張都不會留」。金管會所謂的「附條件」核准，條件只有兩個，第一

In [40]:
df1["內容"]=cleaned
df1

,日期,標題,內容連結,內容
0,2021-06-02,吳東亮終於可分手彰銀！「台新金」將以三招擺脫16年併購包袱,https://www.gvm.com.tw/article/79966,多年來，台新金控董事長吳東亮曾設法啟動併購、更名，甚至不惜全盤放棄彰銀，卻始終無法如願，無論...
1,2021-06-02,本土疫情爆發下，首當其衝的餐飲、百貨股能翻身嗎？,https://www.gvm.com.tw/article/79950,但筆者慶德在進入本文財經分析前，仍是得誠心提醒《遠見》專欄親愛的讀者與全體國人，防疫不分你我...
2,2021-06-01,ESG意識抬頭，近7成台灣女性想用投資讓世界變美好,https://www.gvm.com.tw/article/79815,2020~2021年，無疑是人們展現永續意識覺醒的重要時刻。根據晨星最新報告，全球以永續發展...
3,2021-06-01,金管會點頭！台新金併保德信核准，承諾六年內處分彰銀持股,https://www.gvm.com.tw/article/79945,金管會今天宣布，已於5月31日同意台新金取得保德信人壽100%股份申請案，後續台新金須依所報...
4,2021-06-01,全球經濟邁向多速復甦 價值回歸投資成王道,https://www.gvm.com.tw/article/79636,美國祭出強而有力的財政刺激方案，並加速肺炎疫苗接種，花旗集團觀察，此舉有助於提振全球投資者對...
...,...,...,...,...
552,2020-01-03,端盤子出身的理財天后！蘇西．歐曼走向財務自由的9個步驟,https://www.gvm.com.tw/article/70331,她出生於芝加哥，畢業於伊利諾大學。在完成大學學業後，她搬到了加州的柏克萊。最早她在餐廳當服務...
553,2020-01-02,換一顆「金頭腦」迎接2020,https://www.gvm.com.tw/article/70255,2020來了，你準備好換上一顆「金頭腦」了嗎？一年伊始，正適合盤點人生各階段需求，啟動新年度...
554,2020-01-02,沈榮津︰有感需要時間 2020、2021是爆發年,https://www.gvm.com.tw/article/70294,受中美貿易戰影響，累計台商返鄉登記投資額逾7000億，預計未來三年創造就業人數5萬8322人...
555,2020-01-02,外商聚焦五大產業 台灣哪裡比人強？,https://www.gvm.com.tw/article/70293,眼下，外商在台灣的投資進行式，正在改寫台灣一度敬陪亞洲末座的投資環境。南部科學園區2019年...


In [41]:
import datetime

def get_week_of_month(year, month, day):
    begin = int(datetime.date(year, month, 1).strftime("%W"))
    end = int(datetime.date(year, month, day).strftime("%W"))

    return end - begin + 1

In [42]:
week_id=[]
for time in df1["日期"]:
    y=int(time[0:4])
    m=int(time[5:7])
    d=int(time[8:10])
    week_index=get_week_of_month(y,m,d)
    week_id.append(time[0:4]+"-"+time[5:7]+"-"+str(week_index))
week_id

['2021-06-1',
 '2021-06-1',
 '2021-06-1',
 '2021-06-1',
 '2021-06-1',
 '2021-05-5',
 '2021-05-5',
 '2021-05-5',
 '2021-05-5',
 '2021-05-5',
 '2021-05-5',
 '2021-05-5',
 '2021-05-5',
 '2021-05-5',
 '2021-05-5',
 '2021-05-4',
 '2021-05-4',
 '2021-05-4',
 '2021-05-4',
 '2021-05-4',
 '2021-05-4',
 '2021-05-4',
 '2021-05-4',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-3',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-2',
 '2021-05-1',
 '2021-05-1',
 '2021-05-1',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-5',
 '2021-04-4',
 '2021-04-4',
 '2021-04-4',
 '2021

In [44]:
df1["週期ID"]=week_id
df1

,日期,標題,內容連結,內容,週期ID
0,2021-06-02,吳東亮終於可分手彰銀！「台新金」將以三招擺脫16年併購包袱,https://www.gvm.com.tw/article/79966,多年來，台新金控董事長吳東亮曾設法啟動併購、更名，甚至不惜全盤放棄彰銀，卻始終無法如願，無論...,2021-06-1
1,2021-06-02,本土疫情爆發下，首當其衝的餐飲、百貨股能翻身嗎？,https://www.gvm.com.tw/article/79950,但筆者慶德在進入本文財經分析前，仍是得誠心提醒《遠見》專欄親愛的讀者與全體國人，防疫不分你我...,2021-06-1
2,2021-06-01,ESG意識抬頭，近7成台灣女性想用投資讓世界變美好,https://www.gvm.com.tw/article/79815,2020~2021年，無疑是人們展現永續意識覺醒的重要時刻。根據晨星最新報告，全球以永續發展...,2021-06-1
3,2021-06-01,金管會點頭！台新金併保德信核准，承諾六年內處分彰銀持股,https://www.gvm.com.tw/article/79945,金管會今天宣布，已於5月31日同意台新金取得保德信人壽100%股份申請案，後續台新金須依所報...,2021-06-1
4,2021-06-01,全球經濟邁向多速復甦 價值回歸投資成王道,https://www.gvm.com.tw/article/79636,美國祭出強而有力的財政刺激方案，並加速肺炎疫苗接種，花旗集團觀察，此舉有助於提振全球投資者對...,2021-06-1
...,...,...,...,...,...
552,2020-01-03,端盤子出身的理財天后！蘇西．歐曼走向財務自由的9個步驟,https://www.gvm.com.tw/article/70331,她出生於芝加哥，畢業於伊利諾大學。在完成大學學業後，她搬到了加州的柏克萊。最早她在餐廳當服務...,2020-01-1
553,2020-01-02,換一顆「金頭腦」迎接2020,https://www.gvm.com.tw/article/70255,2020來了，你準備好換上一顆「金頭腦」了嗎？一年伊始，正適合盤點人生各階段需求，啟動新年度...,2020-01-1
554,2020-01-02,沈榮津︰有感需要時間 2020、2021是爆發年,https://www.gvm.com.tw/article/70294,受中美貿易戰影響，累計台商返鄉登記投資額逾7000億，預計未來三年創造就業人數5萬8322人...,2020-01-1
555,2020-01-02,外商聚焦五大產業 台灣哪裡比人強？,https://www.gvm.com.tw/article/70293,眼下，外商在台灣的投資進行式，正在改寫台灣一度敬陪亞洲末座的投資環境。南部科學園區2019年...,2020-01-1


In [45]:
df1.to_csv('遠見(新聞)內文資料.csv',encoding='utf_8_sig')

In [12]:
list2=[]
groups = df3.groupby(df3["週期"])
for i in a:
    ms_df = groups.get_group(i)
    b="".join(ms_df["內容"].tolist())  
    list2.append(b)
len(list2)

22

In [13]:
df4= pd.DataFrame(
{
    '週期':a,
    '內容':list2
})

df4.reset_index(inplace=True)
df4=df4.drop(columns=['index'])
df4

,週期,內容
0,2021-05-5,26日包含華南銀行中崙分行、彰化銀行晴光分行、凱基銀行板橋分行、日盛銀行土城分行均有員工確診...
1,2021-05-4,5、6月股東會旺季前一向是金融股存股最高峰，因受新冠疫情影響，下周一起所有公司停開股東會，不...
2,2021-05-3,法人認為，台股上週週線重摔1457點，除了消息利空帶來的恐慌性因素，部分強勢類股也出現修正，...
3,2021-05-2,根據最新實價揭露，日系藥妝松本清東湖店店面今年3月交易，總坪數約136坪，總價1.37億元，...
4,2021-05-1,今年，現年90歲的巴菲特（Warren Buffett）總算合體黃金拍檔、波克夏的副主席查理...
5,2021-04-5,中信房屋研展室副理張漢超表示，木柵線從動物園站到中山國中站，是國內第一條捷運，內湖線通車後，...
6,2021-04-4,中經院經濟展望中心主任彭素玲指出，全球經濟復甦，對科技產品的創新、應用增強，國內廠商多有擴大...
7,2021-04-3,你手上的現金會隨時流動，意味著你可以立刻動用在花費上。假如你剛領薪，你或許有很多現金可動支。...
8,2021-04-2,「不過很多人只要收入一增加，就立刻變得奢侈，提高生活花費，這也是沒有會計力的表現。」「你的意...
9,2021-04-1,什麼樣的狀態，才叫退休？其實沒有標準答案。自嘲「被迫退休」的理財專家夏韻芬認為，退休要有財務...


In [14]:
import nltk
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba

In [17]:
list4=[]

f = open(r"stopwords_TW.txt",'r',encoding = 'UTF-8')
stopwords_n = f.readlines()
f.close()  
stopwords = [sw.strip().replace('\n','') for sw in stopwords_n]
jieba.set_dictionary('dict_TW.txt')
wordlist=[]

for i in range(5):
    words=jieba.cut(df4["內容"][i], cut_all=False)
    wordlist.clear()
    for word in words:
        wordlist.append(word.replace(" ",""))
        no_punctuation=[x for x in wordlist if not x in stopwords]
    count=Counter(no_punctuation)
    list4.append(count.most_common(10))

Building prefix dict from C:\Users\88696\dict_TW.txt ...
Loading model from cache C:\Users\88696\AppData\Local\Temp\jieba.ud97a2ba90f29c356bf84edcfb304325d.cache
Loading model cost 0.843 seconds.
Prefix dict has been built successfully.


In [18]:
list4

[[('疫情', 28),
  ('確診', 24),
  ('防疫', 21),
  ('保單', 17),
  ('分行', 16),
  ('員工', 16),
  ('隔離', 16),
  ('疫苗', 16),
  ('台灣', 15),
  ('月', 13)],
 [('防疫', 35),
  ('疫情', 33),
  ('股東會', 28),
  ('股東', 28),
  ('公司', 24),
  ('台股', 22),
  ('月', 20),
  ('日', 19),
  ('點', 18),
  ('後', 16)],
 [('台股', 42),
  ('貨幣', 41),
  ('疫苗', 40),
  ('年', 34),
  ('比特幣', 33),
  ('元', 32),
  ('月', 31),
  ('中', 30),
  ('退休', 30),
  ('認為', 27)],
 [('月', 53),
  ('年', 33),
  ('今年', 28),
  ('台股', 26),
  ('美國', 25),
  ('買', 23),
  ('基金', 23),
  ('投資', 22),
  ('市場', 21),
  ('萬', 21)],
 [('公司', 20),
  ('買', 17),
  ('巴菲特', 15),
  ('年', 14),
  ('說', 14),
  ('投資', 13),
  ('波克夏', 11),
  ('崴', 9),
  ('房子', 9),
  ('蒙格', 8)]]

In [19]:
print(count)

Counter({'公司': 20, '買': 17, '巴菲特': 15, '年': 14, '說': 14, '投資': 13, '波克夏': 11, '崴': 9, '房子': 9, '蒙格': 8, '現在': 8, '比特幣': 8, '股票': 8, '應該': 8, '店面': 8, '元': 7, '批評': 7, '已經': 7, '回購': 7, '公設': 7, '股東會': 6, '前': 6, '億美': 6, '後': 6, '道': 6, '提到': 6, '投機': 6, '爺': 6, '大學': 6, '賺': 5, 'SPAC': 5, '好': 5, '部分': 5, '一下': 5, '羅': 5, '姐': 5, '內湖': 5, '房地產': 5, '內政部': 5, '今年': 4, '相關': 4, '20': 4, '榜單': 4, '強調': 4, '一種': 4, '年代': 4, '未來': 4, '問題': 4, '2030': 4, '房': 4, '分享': 4, '買房': 4, '附近': 4, '錢': 4, '經驗': 4, '表示': 4, '住戶': 4, '合理': 4, '黃': 3, '利益': 3, '亞伯': 3, '是要': 3, '最': 3, '發展': 3, '股東': 3, '世界': 3, '企業': 3, '認為': 3, '簡單': 3, '警告': 3, '一個': 3, '擁有': 3, '能源': 3, '巴黎': 3, '氣候': 3, '減排': 3, '提出': 3, '座': 3, '目': 3, '持續': 3, '喜歡': 3, '時': 3, '做': 3, '萬': 3, '當時': 3, '套房': 3, '出租': 3, '房東': 3, '年輕': 3, '拿來': 3, '保值': 3, '改革': 3, '空間': 3, '車位': 3, '花敬': 3, '群表示': 3, '免計': 3, '容積': 3, '檢討': 3, '上限': 3, '金': 2, '出席': 2, '去年': 2, '同期': 2, '已': 2, '非': 2, '業務': 2, '副董事長': 2, '投資人': 2, '全球': 2, '經濟':

In [45]:
df4["內容"][4]

'今年，現年90歲的巴菲特（Warren Buffett）總算合體黃金拍檔、波克夏的副主席查理蒙格（Charlie Munger）。一同出席並主持波克夏的年度股東會（Berkshire Hathaway Annual Meeting of Stockholders）。在股東會前，波克夏發布了今年第一季的營運概況，大賺117億美元（約新台幣3276億元），相較於去年同期虧損497億美元，已大力扭轉頹勢。營業利益也從前一年同期的58.7億美元增至70.2億美元。亮眼成績讓巴菲特在股東會前挽回顏面、重回「股神」地位。出席的還有，巴菲特接班候選人的吉恩（Ajit Jain），以及波克夏公司非保險業務副董事長的亞伯（Greg Abel）。組隊黃金陣容就是要給投資人最完整的全球經濟、公司發展等分析，以及回答股東相關的提問。年會開始，巴菲特列出當今世界上最大的前20家公司，並問道從現在起30年後，還有多少家公司會進入該榜單？當前20大企業的榜單上包括蘋果、沙烏地阿拉伯國家石油公司、微軟、亞馬遜、Alphabet和Facebook。「1989年的前20大公司中，沒有一家在今天的榜單上。」巴菲特提醒。30多年前，全球最大的公司中有一半以上是日本公司，包括銀行和工業公司。榜單上僅有的美國公司是埃克森美孚、奇異、默克集團等。「這告訴你資本主義的運轉非常好，特別是對資本家。世界可以以非常劇烈的方式變化。」巴菲特強調。並補充說道：最好的投資方式是通過指數型基金（ETF）。「不要認為投資是一種致富的簡單途徑，」巴菲特警告投資新手。他強調，挑選大公司比選擇一個有前途的行業要複雜得多，選股不只是弄清楚哪個行業會是有前景的領域，那麼簡單而已。巴菲特提到：在1900年代，大約有2000家公司進入了汽車行業。因為投資者和企業家期望該行業擁有美好的未來。但是在2009年，僅剩下三家汽車製造商，而其中兩家破產。面對外界對化石燃料公司的批評及輿論，針對這點巴菲特也做出回應，認為「擁有雪佛龍（Chevron）沒有任何問題，並期望世界轉向更多可再生能源，波克夏也將在未來對社會造福。」「波克夏的能源公司會在2030年完全達到《巴黎氣候協定》的減排承諾。」曾擔任波克夏能源業務主管的現任副董事長亞伯強調。他並提到美國總統拜登（Joe Biden）上任之後要重返《巴黎氣候協定》，現在的政府也提出排放目標是在2030年減排

In [39]:
len(df4["內容"])

22

In [151]:
def str_count2(str):
    i=0
    for s in str:
        if '\u4e00' <= s <= '\u9fff':
            i+=1
    return i
def str_remove(list_object):
    for word in list_object:
        if str_count2(word) <= 30:
            list_object.remove(word)
    return list_object


removed_sentence=[]
for x in remove_cleaned:
    removed_sentence.append(str_remove(x))
removed_sentence

[['以增幅來看，台北市以大安區增幅近二成最多，新北則是最先爆出社區傳染的蘆洲區增加33%最多，其次為板橋31%，永和30%。',
  '另外，近日確診人數急速上升的三重、新莊，中和，新增待租店面數也快速增加。',
  '屋比統計也發現，雙北店面平均租金開價下修約3%～4％，網路上出現大量「疫情優惠專案」訴求，房東明顯放軟姿態，尋求租客。',
  '屋比房屋總監陳傑鳴表示，這波疫情，對雙北各大商圈不只是再一次重擊，對許多家店家而言，更可能是致命打擊。',
  '他表示，店家少了國際觀光客，本來還有內需市場支持，在斷一隻腳狀況下，勉強還可以緩步前進。但現在肺炎疫情蔓延，本地消費人潮也不見了，商家現在有如雙腳齊斷，只能用爬的苟延殘喘。',
  '尤其是確診人數較多的板橋、萬華、新莊、三重、蘆洲、還有高租金的大安、中山等地區，近期店家營運壓力都相當大，每天開門營業幾乎都要乾燒資金數萬到數十萬不等。',
  '以西門町來說，平均店面每坪月租金開價仍約有每坪8000元上下，坪數約30～40坪的店面商家，每月租金就要約30萬，若加計人力、水電等成本，等於每天開店都要燒掉上萬塊錢，經營壓力相當大。',
  '陳傑鳴表示，目前這些區域空租狀況都有攀升趨勢，若沒客人的日子拉長，商圈店家撐不住的狀況將越來越多。'],
 ['台灣疫情於5月初爆發，新增確診突破百人，雙北防疫在5／17升級至第三級警戒，引發台股大幅修正，全台也於本日（5／19）進入第三級警戒，大家最關注的，莫非是疫情何時可以出現緩解。綜觀全球疫情自去年以來，歐美國家陸續出現二爆及三爆，印度疫情更是在近期大規模爆發，M平方以美國、歐洲主要國家、日本及印度作為參考，統計從政府開始升級防疫、至管控嚴密的時期，與新增確診見高的時間點作為參考。',
  '從其餘國家的經驗可以發現，防疫開始升級後至確診高峰平均歷時二～三個月，而疫情管控指數的高峰與新增確診人數的高峰則相差0.5～一個月，差別考量在於政府是否立即性的提升防疫措施、人口密度、疫苗施打、以及是否會有變種病毒的狀況。以目前全面停止上課、並升級至全國三級警戒下，政府可算是立即性的有所作為，不過，現階段台灣僅進口AZ疫苗，施打劑數佔總人口比率不到1%，下一批AZ疫苗於本日（5／19） 到貨，Mordena疫苗也預計5月底至6月將到貨，7月將會有國產疫苗，因此預估本次疫情高峰尚未到